In [224]:
import pandas as pd 
from tqdm import tqdm 

data = pd.read_csv('../data/enron_emails.csv')

cols = [ 'Message-ID:', 'Date:', 'From:', 'To:', 'Subject:','Mime-Version:', 'Content-Type:',\
    'Content-Transfer-Encoding:', 'X-From:', 'X-To:', 'X-cc:', 'X-bcc:', "X-Folder:", 'X-Origin:',\
    'X-FileName:']

dictts = []
for i in tqdm(range(len(data))):
    row = data.iloc[i]
    parts = row['message'].split('\n')
    dictt = {}
    body = ''
    for part in parts:
        if sum([part.startswith(col) for col in cols]) > 0:
            a, b = part.split(':', 1)
            dictt[a.strip()] = b.strip()
        else:
            body = body + " "+part
    dictt['body'] = body
    dictts.append(dictt)
    
enron = pd.DataFrame(dictts)#.dropna(axis=1)
enron.to_csv('../data/enron_clean.csv')

100%|███████████████████████████████████████████████████████████| 517401/517401 [02:08<00:00, 4030.81it/s]


<img src="https://static01.nyt.com/images/2006/10/24/business/20061024_ENRON_GRAPHIC.gif">

In [483]:
enron = pd.read_csv('../data/enron_clean.csv')

subset         = enron[['X-From', 'X-To', 'Date', 'body']]
subset.columns = ['from', 'to', 'date', 'body']

# subset         = subset.sample(1000)

subset         = subset.dropna()

subset['from'] = subset['from'].apply(lambda x:     x[:x.find('<')].strip() if x.find('<')!= 0 else x)
subset['to']   = subset['to'].apply(  lambda x:     x[:x.find('<')].strip() if x.find('<')!= 0 else x)
subset         = subset[subset.apply(lambda x:   ('@' not in x['from']) and ('@' not in x['to']), axis=1)]
subset         = subset[subset['to'].apply(lambda x: ('(E-mail)' not in x) and (x.count(',') < 2))]
subset         = subset[subset['to'] != '']

execs          = ['Fastow, Andrew', 'Skilling, Jeff', 'Glisan, Ben', 'Delainey, David']
to_map         = subset['to'].apply(lambda x: x in execs)
from_map       = subset['from'].apply(lambda x: x in execs)

subset         = subset[to_map | from_map]

subset.index   = range(len(subset))

subset.shape

(549, 4)

In [484]:
subset

,from,to,date,body
0,"Gahn, Scott","Delainey, David","Fri, 5 Oct 2001 06:48:25 -0700 (PDT)","Cc: jeremy.blachman@enron.com, sean.holmes@en..."
1,"Beck, Sally","Delainey, David","Mon, 4 Jun 2001 12:37:46 -0700 (PDT)","\tm..presto@enron.com, don.black@enron.com, r..."
2,"Beck, Sally","Delainey, David","Mon, 1 Oct 2001 09:50:20 -0700 (PDT)",Thanks for the note and your words of encour...
3,�m ��,"Fastow, Andrew","Mon, 22 Oct 2001 04:57:09 -0700 (PDT)","\tmark.frevert@enron.com, greg.whalley@enron...."
4,"Tribolet, Michael","Delainey, David","Tue, 2 Oct 2001 16:40:24 -0700 (PDT)","\tlisa.mellencamp@enron.com, jeff.golden@enro..."
...,...,...,...,...
544,"Whalley, Greg","Skilling, Jeff","Wed, 1 Aug 2001 21:50:59 -0700 (PDT)",i am in london next week. are you using the...
545,"Whalley, Greg","Delainey, David","Fri, 6 Jul 2001 07:58:22 -0700 (PDT)",i talked with sanders yesterday about our re...
546,"Whalley, Greg","Delainey, David","Tue, 26 Jun 2001 14:06:03 -0700 (PDT)",do you know that your receivables have been ...
547,"Taylor, Liz M.","Delainey, David","Mon, 30 Jul 2001 06:56:43 -0700 (PDT)","\tjanet.dietrich@enron.com, j..kean@enron.com..."


In [485]:
subset['date']      = pd.to_datetime(subset['date'], errors='coerce')
subset              = subset.dropna(subset=['date'])

subset['year']      = subset['date'].apply(lambda x: x.year)
subset['month']     = subset['date'].apply(lambda x: x.month)
subset['day_month'] = subset['date'].apply(lambda x: x.day)
subset['day_week']  = subset['date'].apply(lambda x: x.weekday())
subset['time_hrs']  = subset['date'].apply(lambda x: x.hour)
subset['time_mins'] = subset['date'].apply(lambda x: x.minute)

subset              = subset[subset['year'].apply(lambda x: x in range(1996, 2008))]

del subset['date']

In [486]:
vocab = ' '.join(subset['body'].values)
vocab = vocab.split()
vocab = list(set(vocab))

In [487]:
vocab = [word for word in vocab if '@' not in word]
vocab = [word for word in vocab if '>' not in word]
vocab = [word for word in vocab if '/' not in word]
vocab = [word for word in vocab if ')' not in word]
vocab = [word for word in vocab if '(' not in word]
vocab = [word for word in vocab if '""' not in word]
vocab = [word for word in vocab if "''" not in word]
vocab = [word for word in vocab if '=' not in word]
vocab = [word for word in vocab if word.isnumeric() == False]
vocab = [word for word in vocab if "." not in word]
vocab = [word for word in vocab if "?" not in word]
vocab = [word for word in vocab if "-" not in word]
vocab = [word for word in vocab if "$" not in word]
vocab = [word for word in vocab if "*" not in word]
vocab = [word for word in vocab if word.isalpha()]
vocab = [word for word in vocab if len(word)>3]

vocab = [word.lower() for word in vocab]

vocab = list(set(vocab))

In [488]:
len(vocab)

8251

In [489]:
from tqdm import tqdm 

bow = pd.DataFrame(columns=vocab)
for word in tqdm(vocab): 
    bow[word] = subset['body'].apply(lambda x: x.count(word))

100%|███████████████████████████████████████████████████████████████████████| 8251/8251 [01:36<00:00, 85.64it/s]


In [490]:
a = subset[['from', 'to']]
a.columns = ['FROM', 'TO']
b = pd.concat([a, bow], axis=1)

In [498]:
b = b.set_index(['FROM', 'TO'])
b.to_csv('../data/enron_bow.csv')

In [337]:
subset['month2'] = subset['month'].replace({1: "Jan", 2: "Feb", 3:"March", 4: "April", 5: "May", 6: "June", 7: "July",\
                        8: "Aug", 9: "Sept", 10: "Oct", 11: "Nov", 12: "Dec"})

In [359]:
a.apply(lambda x: ''.join(x.values*x.index), axis=1)

4624      April
380529      Aug
485252     June
227940     Sept
365711      Oct
          ...  
498299      Feb
266653      Jan
431526      Oct
344664     Sept
167314      Oct
Length: 631, dtype: object

In [352]:
a = pd.get_dummies(subset['month2'])

a.apply(lambda x: x.index[x==x.max()][0], axis=1)

4624      April
380529      Aug
485252     June
227940     Sept
365711      Oct
          ...  
498299      Feb
266653      Jan
431526      Oct
344664     Sept
167314      Oct
Length: 631, dtype: object

In [364]:
subset.head()

,from,to,body,year,month,day_month,day_week,time_hrs,time_mins,month2
4624,John Arnol,Kim War,australia definitely sounds cool. might be ...,2001,4,11,2,3,43,April
380529,Mike J Mille,"Don Miller, Benjamin Roger",The following is the best contact informatio...,2000,8,30,2,4,50,Aug
485252,Mark Taylo,Tana Jone,Can you help them with this? ----- Forwarded...,2001,6,5,1,2,6,June
227940,Enron Announcement,All Enron Worldwid,DealBench Enron Net Works' newest initiativ...,2000,9,29,4,11,4,Sept
365711,"Presto, Kevin M.","Roan, Michael",Looks fine. -----Original Message----- Sen...,2001,10,16,1,10,27,Oct


In [479]:
subset = subset[['from', 'to', 'year', 'month', 'day_month', 'day_week', 'time_hrs', 'time_mins']] # 'body'

In [480]:
subset

,from,to,year,month,day_month,day_week,time_hrs,time_mins
0,"Gahn, Scott","Delainey, David",2001,10,5,4,6,48
1,"Beck, Sally","Delainey, David",2001,6,4,0,12,37
2,"Beck, Sally","Delainey, David",2001,10,1,0,9,50
3,�m ��,"Fastow, Andrew",2001,10,22,0,4,57
4,"Tribolet, Michael","Delainey, David",2001,10,2,1,16,40
...,...,...,...,...,...,...,...,...
544,"Whalley, Greg","Skilling, Jeff",2001,8,1,2,21,50
545,"Whalley, Greg","Delainey, David",2001,7,6,4,7,58
546,"Whalley, Greg","Delainey, David",2001,6,26,1,14,6
547,"Taylor, Liz M.","Delainey, David",2001,7,30,0,6,56


In [481]:
subset.to_csv('../data/enron_clean_sample.csv')

In [443]:
subset.head()

,from,to,body,year,month,day_month,day_week,time_hrs,time_mins
0,"Gahn, Scott","Delainey, David","Cc: jeremy.blachman@enron.com, sean.holmes@en...",2001,10,5,4,6,48
1,"Beck, Sally","Delainey, David","\tm..presto@enron.com, don.black@enron.com, r...",2001,6,4,0,12,37
2,"Beck, Sally","Delainey, David",Thanks for the note and your words of encour...,2001,10,1,0,9,50
3,�m ��,"Fastow, Andrew","\tmark.frevert@enron.com, greg.whalley@enron....",2001,10,22,0,4,57
4,"Tribolet, Michael","Delainey, David","\tlisa.mellencamp@enron.com, jeff.golden@enro...",2001,10,2,1,16,40


In [445]:
all_people = pd.concat([subset['from'], subset['to']]).drop_duplicates()

matrix = pd.DataFrame(0, columns=all_people, index=all_people)

for i in range(len(subset)):
  row = subset.iloc[i]
  matrix.loc[row['from'], row['to']] = matrix.loc[row['from'], row['to']] + 1
    
matrix.index.name = 'from'
matrix.columns.name = 'to'

In [476]:
matrix

to,"Gahn, Scott","Beck, Sally",�m ��,"Tribolet, Michael","Glisan, Ben","Baker, Ron","Corteselli, Gina","Rieker, Paula","Schultz, Cassandra","Sczykutowicz, Marion",...,"Tycholiz, Barry","Taylor, Liz M.","Leff, Dan","Muller, Mark","Allen, Phillip K.","Seyfried, Bryan","Dietrich, Janet","Ambler, John","Fernandez, Fernando","Lavorato, John"
from,,,,,,,,,,,,,,,,,,,,,
"Gahn, Scott",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Beck, Sally",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
�m ��,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Tribolet, Michael",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Glisan, Ben",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Seyfried, Bryan",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Dietrich, Janet",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Ambler, John",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [499]:
matrix.to_csv('../data/enron_adjacency.csv')

In [474]:

sents = matrix.sum(axis=1)
received = matrix.sum()
pd.concat([sents, received], axis=1).sort_values(1)

,0,1
"Gahn, Scott",1,0
"""Jan_Levine""",2,0
"""Mendenhall, Patrick M.""",1,0
"Neubauer, Dave",1,0
"Versacci, Timothy",1,0
...,...,...
"Allen, Phillip K.",0,3
"Glisan, Ben",3,9
"Fastow, Andrew",4,11
"Delainey, David",9,85
